# Setup the project

In [ ]:
import mlrun
from mlrun import get_or_create_project

project_name = "job-failure-test-project"
project = get_or_create_project(project_name)


# Create an alert

In [ ]:
notifications = [
            {
                "kind": "slack",
                "name": "",
                "message": "Running a job has failed",
                "severity": "warning",
                "when": ["now"],
                "condition": "failed",
                "secret_params": {
                    "webhook": "https://hooks.slack.com",
                },
            }
                ]
alert_data = mlrun.common.schemas.AlertConfig(
            project=project_name,
            name="failure",
            summary="Running a job has failed",
            severity="low",
            entity={"kind": "job", "project": project_name, "id": "*"},
            trigger={"events": ["failed"]},
            criteria=None,
            notifications=notifications,
        ).dict()

project.create_alert_config("failed", alert_data)

In [ ]:
%%writefile fn.py
import mlrun

@mlrun.handler()
def fail_function(context):
    raise Exception("This function intentionally fails")


# Create a job and make it fail

In [ ]:
!pip install pytest
import pytest

function = mlrun.code_to_function(
            name="test-func",
            project=project_name,
            filename='fn.py',
            handler="fail_function",
            kind="job",
            image="docker.io/yacouby1/mlrun:unstable3",
        )
function.set_image_pull_configuration(image_pull_policy="Always")
task = mlrun.new_task()
function.run(task)
